# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 10 2022 2:06PM,243899,20,8384543,"Exact-Rx, Inc.",Released
1,Aug 10 2022 2:06PM,243899,20,8385777,"Exact-Rx, Inc.",Released
2,Aug 10 2022 2:06PM,243898,10,0085932335,ISDIN Corporation,Released
3,Aug 10 2022 2:06PM,243898,10,0085932340,ISDIN Corporation,Released
4,Aug 10 2022 2:06PM,243898,10,0085932342,ISDIN Corporation,Released
5,Aug 10 2022 2:06PM,243898,10,0085932338,ISDIN Corporation,Released
6,Aug 10 2022 2:06PM,243898,10,0085932339,ISDIN Corporation,Released
7,Aug 10 2022 2:06PM,243898,10,0085932341,ISDIN Corporation,Released
8,Aug 10 2022 2:06PM,243898,10,0085932783,ISDIN Corporation,Released
9,Aug 10 2022 2:06PM,243898,10,0085932814,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,243895,Released,6
32,243896,Released,14
33,243897,Released,1
34,243898,Released,28
35,243899,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243895,NaN,NaN,6.0
243896,NaN,NaN,14.0
243897,NaN,NaN,1.0
243898,NaN,NaN,28.0
243899,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243805,0.0,12.0,3.0
243808,32.0,18.0,5.0
243811,0.0,0.0,3.0
243828,0.0,1.0,0.0
243846,0.0,8.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243805,0,12,3
243808,32,18,5
243811,0,0,3
243828,0,1,0
243846,0,8,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243805,0,12,3
1,243808,32,18,5
2,243811,0,0,3
3,243828,0,1,0
4,243846,0,8,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243805,,12,3
1,243808,32,18,5
2,243811,,,3
3,243828,,1,
4,243846,,8,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 10 2022 2:06PM,243899,20,"Exact-Rx, Inc."
2,Aug 10 2022 2:06PM,243898,10,ISDIN Corporation
30,Aug 10 2022 2:04PM,243897,21,"NBTY Global, Inc."
31,Aug 10 2022 1:53PM,243895,10,"Methapharm, Inc."
37,Aug 10 2022 1:53PM,243894,10,"Methapharm, Inc."
41,Aug 10 2022 1:51PM,243896,19,"AdvaGen Pharma, Ltd"
55,Aug 10 2022 1:13PM,243891,19,"AdvaGen Pharma, Ltd"
58,Aug 10 2022 1:10PM,243890,10,"Snap Medical Industries, LLC"
61,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc."
62,Aug 10 2022 12:55PM,243887,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 10 2022 2:06PM,243899,20,"Exact-Rx, Inc.",,,2
1,Aug 10 2022 2:06PM,243898,10,ISDIN Corporation,,,28
2,Aug 10 2022 2:04PM,243897,21,"NBTY Global, Inc.",,,1
3,Aug 10 2022 1:53PM,243895,10,"Methapharm, Inc.",,,6
4,Aug 10 2022 1:53PM,243894,10,"Methapharm, Inc.",,,4
5,Aug 10 2022 1:51PM,243896,19,"AdvaGen Pharma, Ltd",,,14
6,Aug 10 2022 1:13PM,243891,19,"AdvaGen Pharma, Ltd",,,3
7,Aug 10 2022 1:10PM,243890,10,"Snap Medical Industries, LLC",,,3
8,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc.",,,1
9,Aug 10 2022 12:55PM,243887,10,Emerginnova,,,7


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 2:06PM,243899,20,"Exact-Rx, Inc.",2,,
1,Aug 10 2022 2:06PM,243898,10,ISDIN Corporation,28,,
2,Aug 10 2022 2:04PM,243897,21,"NBTY Global, Inc.",1,,
3,Aug 10 2022 1:53PM,243895,10,"Methapharm, Inc.",6,,
4,Aug 10 2022 1:53PM,243894,10,"Methapharm, Inc.",4,,
5,Aug 10 2022 1:51PM,243896,19,"AdvaGen Pharma, Ltd",14,,
6,Aug 10 2022 1:13PM,243891,19,"AdvaGen Pharma, Ltd",3,,
7,Aug 10 2022 1:10PM,243890,10,"Snap Medical Industries, LLC",3,,
8,Aug 10 2022 12:57PM,243888,19,"GUSA Granules USA, Inc.",1,,
9,Aug 10 2022 12:55PM,243887,10,Emerginnova,7,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 2:06PM,243899,20,"Exact-Rx, Inc.",2,,
1,Aug 10 2022 2:06PM,243898,10,ISDIN Corporation,28,,
2,Aug 10 2022 2:04PM,243897,21,"NBTY Global, Inc.",1,,
3,Aug 10 2022 1:53PM,243895,10,"Methapharm, Inc.",6,,
4,Aug 10 2022 1:53PM,243894,10,"Methapharm, Inc.",4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 2:06PM,243899,20,"Exact-Rx, Inc.",2.0,NaN,NaN
1,Aug 10 2022 2:06PM,243898,10,ISDIN Corporation,28.0,NaN,NaN
2,Aug 10 2022 2:04PM,243897,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Aug 10 2022 1:53PM,243895,10,"Methapharm, Inc.",6.0,NaN,NaN
4,Aug 10 2022 1:53PM,243894,10,"Methapharm, Inc.",4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 2:06PM,243899,20,"Exact-Rx, Inc.",2.0,0.0,0.0
1,Aug 10 2022 2:06PM,243898,10,ISDIN Corporation,28.0,0.0,0.0
2,Aug 10 2022 2:04PM,243897,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Aug 10 2022 1:53PM,243895,10,"Methapharm, Inc.",6.0,0.0,0.0
4,Aug 10 2022 1:53PM,243894,10,"Methapharm, Inc.",4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2438752,59.0,21.0,5.0
15,731587,28.0,16.0,0.0
16,243865,1.0,0.0,0.0
19,1707192,21.0,2.0,0.0
20,1219230,43.0,31.0,32.0
21,487725,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2438752,59.0,21.0,5.0
1,15,731587,28.0,16.0,0.0
2,16,243865,1.0,0.0,0.0
3,19,1707192,21.0,2.0,0.0
4,20,1219230,43.0,31.0,32.0
5,21,487725,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,59.0,21.0,5.0
1,15,28.0,16.0,0.0
2,16,1.0,0.0,0.0
3,19,21.0,2.0,0.0
4,20,43.0,31.0,32.0
5,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,59.0
1,15,Released,28.0
2,16,Released,1.0
3,19,Released,21.0
4,20,Released,43.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,5.0,0.0,0.0,0.0,32.0,0.0
Executing,21.0,16.0,0.0,2.0,31.0,1.0
Released,59.0,28.0,1.0,21.0,43.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,32.0,0.0
1,Executing,21.0,16.0,0.0,2.0,31.0,1.0
2,Released,59.0,28.0,1.0,21.0,43.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,32.0,0.0
1,Executing,21.0,16.0,0.0,2.0,31.0,1.0
2,Released,59.0,28.0,1.0,21.0,43.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()